# Phenotype extraction

In this notebook, we demo the first module of the GwasKB system, which extracts the phenotypes that are studied in each paper.

Before starting, make sure you have downloaded all the datasets: the phenotype ontologies, the GWAS Catalog database, and the open-access GWAS papers.

## Preparations

We start by configuring Jupyter and setting up our environment.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import cPickle
import numpy as np
import sqlalchemy

# set the paths to snorkel and gwaskb
sys.path.append('../snorkel-tables')
sys.path.append('../src')
sys.path.append('../src/crawler')

# set up the directory with the input papers
abstract_dir = '../data/db/papers'

# set up matplotlib
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,4)

# create a Snorkel session
from snorkel import SnorkelSession
session = SnorkelSession()

### Load corpus

Our system will read PubMed papers that have been previously identified as GWAS-related. We load this corpus below

In [2]:
from extractor.parser import GWASXMLAbstractParser

xml_parser = GWASXMLAbstractParser(
    path=abstract_dir,
    doc='./*',
    title='.//front//article-title//text()',
    abstract='.//abstract//p//text()',
    par1='.//body/p[1]//text()',
    id='.//article-id[@pub-id-type="pmid"]/text()',
    keep_xml_tree=True)

`GWASXMLAbstractParser` is a custom parser that we wrote. For each paper, it extracts the title and either the abstract of the first paragraph (if there is no abstract).

In [4]:
from snorkel.parser import SentenceParser
from snorkel.parser import CorpusParser
from snorkel.models import Corpus

# this splits documents into sentences and parses each sentence with Stanford CoreNLP
sent_parser = SentenceParser(timeout=600000)

try:
    corpus = session.query(Corpus).filter(Corpus.name == 'GWAS Corpus').one()
except:
    cp = CorpusParser(xml_parser, sent_parser)
    %time corpus = cp.parse_corpus(name='GWAS Corpus', session=session)
    session.add(corpus)
    session.commit()

print 'Loaded corpus of %d documents' % len(corpus)

Loaded corpus of 589 documents


## Candidate extraction

The first stage is to generate a large set of candidate phenotypes, which may or may not be correct. After that, we will train classifiers to predict which ones are correct.

### Extract candidates

We first load our phenotype ontologies, which will be used to generate candidates.

In [5]:
from db.kb import KnowledgeBase
from extractor.util import make_ngrams

# collect phenotype list
kb = KnowledgeBase()

# efo phenotypes
efo_phenotype_list0 = kb.get_phenotype_candidates(source='efo-matching', peek=False) # TODO: remove peaking
efo_phenotype_list = list(make_ngrams(efo_phenotype_list0))
# snomed keywords
snomed_phenotype_list = kb.get_phenotype_candidates(source='snomed')
# mesh diseases
mesh_phenotype_list0 = kb.get_phenotype_candidates(source='mesh')
mesh_phenotype_list = list(make_ngrams(mesh_phenotype_list0))
# mesh chemicals
chem_phenotype_list = kb.get_phenotype_candidates(source='chemical')
# regex matches
rgx = u'[A-Za-z\u2013-]+ (disease|trait|phenotype|outcome|response|quantitative trait|measurement|response|side effects)s?'

We define matchers and an extractor that generate candaites based on these ontologies.

In [6]:
from snorkel.candidates import Ngrams
from snorkel.matchers import DictionaryMatch, Union, RegexMatchSpan
from extractor.matcher import PhenotypeMatcher
from extractor.util import change_name

# Define a candidate space
ngrams = Ngrams(n_max=7)

# Define a matcher for each ontology
efo_phen_matcher = PhenotypeMatcher(d=efo_phenotype_list, ignore_case=True, mod_fn=change_name)
snom_phen_matcher = PhenotypeMatcher(d=snomed_phenotype_list, ignore_case=True, mod_fn=change_name)
mesh_phen_matcher = PhenotypeMatcher(d=mesh_phenotype_list, ignore_case=True, mod_fn=change_name)
chem_phen_matcher = DictionaryMatch(d=chem_phenotype_list, longest_match_only=True, ignore_case=True)
regex_phen_matcher = RegexMatchSpan(rgx=rgx)

# The phenotype matcher is the union of these
phen_matcher = Union(efo_phen_matcher, snom_phen_matcher, mesh_phen_matcher, chem_phen_matcher, regex_phen_matcher)

# Define the extractor
from snorkel.candidates import CandidateExtractor
from snorkel.models import candidate_subclass

Phenotype = candidate_subclass('SnorkelPhenotype', ['phenotype'])
phen_extractor = CandidateExtractor(Phenotype, ngrams, phen_matcher)

Finally, we extract the candidates.

In [7]:
from snorkel.models import CandidateSet

try:
    phen_c = session.query(CandidateSet).filter(CandidateSet.name == 'Phenotype Candidates').one()
except:
    sentences = [s for doc in corpus for s in doc.sentences]
    print '%d sentences loaded' % len(sentences)
    %time phen_c = phen_extractor.extract(sentences, 'Phenotype Candidates', session)
    session.add(phen_c)
    session.commit()

print '%d candidates extracted' % len(phen_c)

71476 candidates extracted


We would like to remove nested candidates as well as obviously wrong candidates.

In [8]:
from extractor.candidates import deduplicate, filter_cand

# we filter candidates or candidates that don't occur within first 3 sentences
# TODO: add stopwords: genome, association, population, analysis
def filter_fn(cand, attrib='phenotype'):
    txt = getattr(cand, attrib).get_span()
    sent_n = getattr(cand, attrib).parent.position
    return False if len(txt) < 5 or sent_n > 2 else True

try:
    new_phen_c = session.query(CandidateSet).filter(CandidateSet.name == 'Filtered Phenotype Candidates').one()
except:
    new_phen_c = CandidateSet(name='Filtered Phenotype Candidates')
    for cand in filter_cand(deduplicate(phen_c), filter_fn=filter_fn):
        new_phen_c.append(cand)
    session.add(new_phen_c)
    session.commit()

print len(phen_c) - len(new_phen_c), 'candidates dropped, now we have', len(new_phen_c)
phen_c = new_phen_c

64008 candidates dropped, now we have 7468


## Learning the correctness of our candidates

Next, we will train machine learning models to identify which phenotype candidates are actually correct.

### Generating a labeled set of examples

We first split data into an (unlabeled) training set (since we will use unsupervised risk estimation to train a candidate on it), and a dev/test set.

In [9]:
try:
    train_c = session.query(CandidateSet).filter(CandidateSet.name == 'Phenotype Training Candidates').one()
    devtest_c = session.query(CandidateSet).filter(CandidateSet.name == 'Phenotype Dev/Test Candidates').one()
except:
    # delete any previous sets with that name
    session.query(CandidateSet).filter(CandidateSet.name == 'Phenotype Training Candidates').delete()
    session.query(CandidateSet).filter(CandidateSet.name == 'Phenotype Dev/Test Candidates').delete()

    frac_test = 0.5

    # initialize the new sets
    train_c = CandidateSet(name='Phenotype Training Candidates')
    devtest_c = CandidateSet(name='Phenotype Dev/Test Candidates')

    # choose a random subset for the labeled set
    n_test = len(phen_c) * frac_test
    test_idx = set(np.random.choice(len(phen_c), size=(n_test,), replace=False))

    # add to the sets
    for i, c in enumerate(phen_c):
        if i in test_idx:
            devtest_c.append(c)
        else:
            train_c.append(c)

    # save the results
    session.add(train_c)
    session.add(devtest_c)
    session.commit()

print 'Initialized %d training and %d dev/testing candidates' % (len(train_c), len(devtest_c))

Initialized 3734 training and 3734 dev/testing candidates


We will come back to this split later on.

### Labeling functions

Following the data programming approach, we define set of labeling functions. We will learn their accuracy via unsupervised learning and use them for classifying candidates.

First, we need to preload data form our phenotype dictionaries, and we also define common stopwords that we will try to filter out.

In [12]:
import re, string
from nltk.stem import PorterStemmer
from db.kb import KnowledgeBase
punctuation = set(string.punctuation)
stemmer = PorterStemmer()

# load set of dictionary phenotypes
kb = KnowledgeBase()
phenotype_list = kb.get_phenotype_candidates() # TODO: load disease names from NCBI
phenotype_list = [phenotype for phenotype in phenotype_list]
phenotype_set = set(phenotype_list)

# load stopwords
with open('../data/phenotypes/snorkel/dicts/manual_stopwords.txt') as f:
    stopwords = {line.strip() for line in f}
stopwords.update(['analysis', 'age', 'drug', 'community', 'detect', 'activity', 'genome',
                  'genetic', 'phenotype', 'response', 'population', 'parameter', 'diagnosis',
                  'level', 'survival', 'maternal', 'paternal', 'clinical', 'joint', 'related',
                  'status', 'risk', 'protein', 'association', 'signal', 'pathway', 'genotype', 'scale',
                  'human', 'family', 'heart', 'general', 'chromosome', 'susceptibility', 'select', 
                  'medical', 'system', 'trait', 'suggest', 'confirm', 'subclinical', 'receptor', 
                  'class', 'adult', 'affecting', 'increase'])
from nltk.corpus import stopwords as nltk_stopwords
stopwords.update(nltk_stopwords.words('english'))
stopwords = {stemmer.stem(word) for word in stopwords}

We also define a few helpers.

In [13]:
def get_phenotype(entity, stem=False):
    phenotype = entity.get_span()
    if stem: phenotype = stemmer.stem(phenotype)
    return phenotype.lower()

def stem_list(L):
    return [stemmer.stem(l.lower()) for l in L]

def span(c):
    return c if isinstance(c, TemporarySpan) else c[-1]

def has_stopwords(m):
    txt = span(m).get_span()
    txt = ''.join(ch for ch in txt if ch not in punctuation)
    words = txt.lower().split()
    return True if all(word in stopwords for word in words) or \
                  all(stemmer.stem(word) in stopwords for word in words) or \
                  all(change_name(word) in stopwords for word in words) else False
        
# candidate_by_sent = dict()
# for c in phen_c:
#     if has_stopwords(c): continue
#     context_id = c[0].parent.document.name, c[0].parent.sentence.position
#     if context_id not in candidate_by_sent:
#         candidate_by_sent[context_id] = [999]
#     candidate_by_sent[context_id].append(c)        

Now we define the functions themselves.

We start with functions that are indicative of the mention being true.

In [14]:
from snorkel.annotations import LabelManager
from snorkel.lf_helpers import *

label_manager = LabelManager()

# positive LFs
def LF_first_sentence(m):
    return +15 if span(m).parent.position == 0 and not has_stopwords(m) else 0
def LF_from_regex(m):
    if span(m).parent.position == 0 and not regex_phen_matcher._f(span(m)) and not LF_bad_words(m): return +5
    else: return 0
def LF_with_acronym(m):
    post_txt = ''.join(right_text(m, attr='words', window=5))
    return +1 if re.search(r'\([A-Z]{2,4}\)', post_txt) else 0
def LF_many_words(m):
    return +1 if len(span(m).get_span().split()) >= 3 else 0
def LF_start_of_sentence(m):
    return +1 if m[0].get_word_start() <= 5 and not has_stopwords(m) and not LF_no_nouns(m) else 0
def LF_first_mention_in_sentence(m):
    context_id = m[0].parent.document.name, m[0].parent.sentence.position
    other_pos = [c.get_word_start() for c in candidate_by_sent[context_id]]
    return +1 if m.get_word_start() == min(other_pos) else 0

LFs_pos = [LF_first_sentence, LF_with_acronym, LF_from_regex, LF_many_words, LF_start_of_sentence]

Next, we also define functions that are indicative of the mention being spurious.

In [15]:
# negative LFs
def LF_bad_words(m):
    bad_words = ['disease', 'single', 'map', 'genetic variation', '( p <']
    return -100 if any(span(m).get_span().lower().startswith(b) for b in bad_words) else 0
def LF_short(m):
    txt = span(m).get_attrib_span('words', 3)
    return -50 if len(txt) < 5 else 0
def LF_no_nouns(m):
    return -10 if not any(t.startswith('NN') for t in span(m).get_attrib_tokens('pos_tags')) else 0
def LF_not_first_sentences(m):
    return -1 if span(m).parent.position > 1 else 0
def LF_stopwords(m):
    return -50 if has_stopwords(m) else 0

LFs_neg = [LF_bad_words, LF_short, LF_no_nouns, LF_not_first_sentences, LF_stopwords]

Finally, we combine both functions and use them to label our training set.

In [21]:
LFs = LFs_pos + LFs_neg

try:
    %time L_train = label_manager.load(session, train_c, 'Phenotype LF Labels')
except sqlalchemy.orm.exc.NoResultFound:
    %time L_train = label_manager.create(session, train_c, 'Phenotype LF Labels', f=LFs)

Generating annotations for 3734 candidates...
[========================================] 100%
Loading sparse Label matrix...
CPU times: user 1min 6s, sys: 778 ms, total: 1min 7s
Wall time: 1min 9s


The table below displays some statistics about our labeling functions.

In [22]:
L_train.lf_stats()

,conflicts,coverage,j,overlaps
LF_first_sentence,0.204874,1.884039,0,1.884039
LF_with_acronym,0.022496,0.036154,1,0.027852
LF_from_regex,0.784681,1.289502,2,1.289502
LF_many_words,0.030262,0.114087,3,0.084092
LF_bad_words,0.615961,1.874665,4,1.874665
LF_short,0.000000,0.000000,5,0.000000
LF_no_nouns,0.728441,3.211034,6,3.211034
LF_not_first_sentences,0.044456,0.339047,7,0.261650
LF_stopwords,8.128013,22.683449,8,22.683449
LF_start_of_sentence,0.017675,0.122389,9,0.075254


### Training machine learning models

So far, we have defined a set of noisy labeling functions that are indicative of each mention being true or false. Next, we use data programming to train a noise-aware generative model that can estimate the accuracies of these labeling functions from data in an unsupervised manner.

The resulting model can be used to provide a learning signal for a second discriminative model (as described in the data programming paper), or it can be used to make predictions directly. Here, we will follow the latter approach.

In [34]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L_train, n_iter=10000, rate=1e-2)

Training marginals (!= 0.5):	3734
Features:			10
Begin training for rate=0.01, mu=1e-06
	Learning epoch = 0	Gradient mag. = 6.173647
	Learning epoch = 250	Gradient mag. = 1.185198
	Learning epoch = 500	Gradient mag. = 0.474470
	Learning epoch = 750	Gradient mag. = 0.376908
	Learning epoch = 1000	Gradient mag. = 0.321143
	Learning epoch = 1250	Gradient mag. = 0.274899
	Learning epoch = 1500	Gradient mag. = 0.235713
	Learning epoch = 1750	Gradient mag. = 0.202393
	Learning epoch = 2000	Gradient mag. = 0.173994
	Learning epoch = 2250	Gradient mag. = 0.149745
	Learning epoch = 2500	Gradient mag. = 0.129007
	Learning epoch = 2750	Gradient mag. = 0.111250
	Learning epoch = 3000	Gradient mag. = 0.096034
	Learning epoch = 3250	Gradient mag. = 0.082986
	Learning epoch = 3500	Gradient mag. = 0.071793
	Learning epoch = 3750	Gradient mag. = 0.062194
	Learning epoch = 4000	Gradient mag. = 0.053963
	Learning epoch = 4250	Gradient mag. = 0.046909
	Learning epoch = 4500	Gradient mag. = 0.040868
	Learn

The numbers below give an estimate of the weights learned for each feature. As you can see, several features have been substantially reweighted (while some others have remained mostly unchanged).

In [35]:
gen_model.w

array([ 3.39941976,  0.81469345, -0.26252601,  0.94585087,  9.96643577,
        0.98503744,  3.47245758,  1.94415812,  9.99916541,  1.09704151])

### Look at results on the test set (optional)

We can adjust the above classifier (e.g. the labelling functions, the hyper-parameters) and assess the results on the dev/test set. This section shows how this can be done.

We will label a small number of dev/test candidates.

In [20]:
n_labeled = 300 # number of candidates to label

random_idx = np.random.choice(len(devtest_c), size=(n_labeled,), replace=False)
labeled_c = CandidateSet(name='Phenotype Labeled Candidates')
for i in random_idx:
    labeled_c.append(devtest_c[i])

We may use the Snorkel viewer to label a set of examples.

In [21]:
from snorkel.viewer import SentenceNgramViewer
sv = SentenceNgramViewer(labeled_c, session, annotator_name="Snorkel Phenotype Annotations")

<IPython.core.display.Javascript object>

This will display the viewer.

In [69]:
sv

We can compare these labels against the predictions of our classifier.

In [70]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

%time Y_test = label_manager.load(session, labeled_c, 'Snorkel Phenotype Annotations')
%time L_test = label_manager.create(session, labeled_c, 'Phenotype LF Test Labels', f=LFs)
# session.commit()

CPU times: user 150 ms, sys: 29.2 ms, total: 179 ms
Wall time: 318 ms
Generating annotations for 300 candidates...
[========================================] 100%
Loading sparse Label matrix...
CPU times: user 6.15 s, sys: 298 ms, total: 6.45 s
Wall time: 7.44 s


In [72]:
gen_model.score(L_test, Y_test, labeled_c)

## Classify all the papers

We now have a classifier that can score phenotype mentions in the text. Let's apply this classifier to assign a phenotype to each of our papers.

### Analyze / Visualize

If a mention occurs in the title, its probably correct, we can take it.

Question: what papers did not have any disease mentions in the title?

In [28]:
from snorkel.annotations import LabelManager, AnnotationKeySet

label_manager = LabelManager()

# delete existing labels
session.rollback()
session.query(AnnotationKeySet).filter(AnnotationKeySet.name == 'Phenotype LF All Labels').delete()
%time L_all = label_manager.create(session, phen_c, 'Phenotype LF All Labels', f=LFs)

Generating annotations for 7468 candidates...
[========================================] 100%
Loading sparse Label matrix...
CPU times: user 1min 50s, sys: 730 ms, total: 1min 50s
Wall time: 1min 51s


Let's now visualize what we found.

In [36]:
scores = gen_model.odds_unw(L_all)
# scores = gen_model.odds_unw(L_all) # this also works well
score_dict = { doc.name : list() for doc in corpus.documents }
for s, c in zip(scores, phen_c):
    score_dict[c[0].parent.document.name].append((s,c))

results = dict()
for pmid, preds in score_dict.items():
    if preds: 
        best_c = sorted(preds, reverse=True)[0][1]
        results[best_c[0].parent.document.name] = best_c
    

In [37]:
for d in corpus.documents[:5]:
    print d, kb.paper_by_pmid(d.name).title
    print unicode(results.get(d.name, None)), [LF(results.get(d.name)) for LF in LFs]
    try:
        print sorted(score_dict[d.name], reverse=True)[:5]
    except UnicodeEncodeError:
        print 'Unicode error'
    print

Document 25086665 Genome-wide association study identifies multiple susceptibility loci for pancreatic cancer.
SnorkelPhenotype(Span("pancreatic cancer", parent=6962, chars=[74,90], words=[8,9])) [15, 0, 5, 0, 0, 0, 0, 0, 0, 0]
[(20.0, SnorkelPhenotype(Span("pancreatic cancer", parent=6962, chars=[74,90], words=[8,9]))), (0.0, SnorkelPhenotype(Span("pancreatic cancer", parent=6963, chars=[96,112], words=[14,15]))), (-1.0, SnorkelPhenotype(Span("ratio", parent=6964, chars=[98,102], words=[16,16]))), (-11.0, SnorkelPhenotype(Span("reached", parent=6964, chars=[14,20], words=[3,3]))), (-45.0, SnorkelPhenotype(Span("association", parent=6962, chars=[12,22], words=[1,1])))]

Document 23349640 Susceptibility loci associated with specific and shared subtypes of lymphoid malignancies.
SnorkelPhenotype(Span("Malignancies", parent=5094, chars=[77,88], words=[10,10])) [15, 0, 5, 0, 0, 0, 0, 0, 0, 0]
[(20.0, SnorkelPhenotype(Span("Malignancies", parent=5094, chars=[77,88], words=[10,10]))), (1.0, 


### Save results

In [39]:
import string
from nltk.corpus import stopwords as nltk_stopwords

punctuation = set(string.punctuation)
nltk_stopword_set = set(nltk_stopwords.words('english'))

def clean_stopwords(txt):
    words = txt.split()
    i = 0
    new_words = []
    while i < len(words):
        i += 1
        if words[i-1] in nltk_stopword_set or words[i-1] in punctuation: continue
        new_word = words[i-1].strip(',.;')
        if new_word.islower() or new_word.isupper(): new_word = new_word.title()
        new_words.append(new_word)
    return ' '.join(new_words)

with open('results/nb-output/phenotypes.extracted.tsv', 'w') as f:
    for d in corpus.documents:
        # pick the top two results:
        best = sorted(score_dict[d.name], reverse=True)[:3]
        # if both are in title, report both, otherwise report only the best one
        if len(best) == 3 and best[2][1][0].parent.position == 0 and best[1][0] - best[2][0] < 3:
            (_, r1), (_, r2), (_, r3) = best
            phen = '|'.join(set([clean_stopwords(r[0].get_span()) for s,r in best[:3]]))
        elif len(best) >= 2 and best[1][1][0].parent.position == 0 and best[1][0] > 5:
            phen = '|'.join(set([clean_stopwords(r[0].get_span()) for s,r in best[:2]]))                
        else:
            phen = clean_stopwords(best[0][1][0].get_span())
        out_str = u'%s\t%s\t\n' % (d.name, phen)        
        f.write(out_str.encode("UTF-8"))
        